## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

## this is not needed
From google.colab import drive
drive.mount('/gdrive')

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('Prices.csv')
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Check all columns in the dataset

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [6]:
drop_attrs = ["date", "symbol"]
data = data.drop(labels=drop_attrs, axis=1)
print("Shape: ", data.shape)

Shape:  (851264, 5)


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 5 columns):
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5)
memory usage: 32.5 MB


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [8]:
prices_df_sample = data.head(1000).copy(deep = True)
print("Shape: ", prices_df_sample.shape)

Shape:  (1000, 5)


In [9]:
prices_df_sample["volume"]=prices_df_sample["volume"]/1000000

In [10]:
prices_df_sample["volume"].head()

0    2.1636
1    2.3864
2    2.4895
3    2.0063
4    1.4086
Name: volume, dtype: float64

### Divide the data into train and test sets

In [11]:
from sklearn.model_selection import train_test_split

#### Convert Training and Test Data to numpy float32 arrays


In [12]:
prices_df_sample=prices_df_sample.values

In [13]:
type(prices_df_sample)

numpy.ndarray

In [14]:
prices_df_sample[:,0:4]

array([[123.43    , 125.839996, 122.309998, 126.25    ],
       [125.239998, 119.980003, 119.940002, 125.540001],
       [116.379997, 114.949997, 114.93    , 119.739998],
       ...,
       [ 28.32    ,  28.77    ,  28.01    ,  28.809999],
       [ 44.      ,  44.799999,  43.75    ,  44.810001],
       [ 36.080002,  37.139999,  36.009998,  37.23    ]])

In [15]:
# split into input (X) and output (Y) variables
X = prices_df_sample[:,0:4]
Y = prices_df_sample[:,4]
# split into 70% for train and 30% for test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [16]:
from sklearn.preprocessing import Normalizer

In [17]:
transformer = Normalizer().fit(X_train)
X_train_n=transformer.transform(X_train)

In [18]:
transformer = Normalizer().fit(X_test)
X_test_n=transformer.transform(X_test)

## Building the Model in tensorflow

In [19]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.nn.l2_normalize(x,1)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [20]:
#Define Loss
def loss(y_actual, y_predicted):   
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

In [21]:
#Define Gradient Descent function
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record math ops on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss w.r.t Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

In [22]:
def prediction(x, w, b):
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    return y


In [23]:
X_train_n.shape

(700, 4)

In [24]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

Instructions for updating:
Colocations handled automatically by placer.


2.Define a function to calculate prediction

In [25]:
y = tf.add(tf.matmul(x_n,W),b,name='output')

3.Loss (Cost) Function [Mean square error]

In [26]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

In [27]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [28]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [29]:
W_array=[]
b_array=[]

In [30]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_train_n, y_:y_train})
    W_array.append(W)
    b_array.append(b)
    
    print ('Training loss at step: ', epoch, ' is ', train_loss)
   ## print ('W at step: ', epoch, ' is ', W_array[2])

Training loss at step:  0  is  305.42902
Training loss at step:  1  is  298.3284
Training loss at step:  2  is  292.8297
Training loss at step:  3  is  288.5715
Training loss at step:  4  is  285.27383
Training loss at step:  5  is  282.72012
Training loss at step:  6  is  280.74274
Training loss at step:  7  is  279.211
Training loss at step:  8  is  278.02515
Training loss at step:  9  is  277.10678
Training loss at step:  10  is  276.39557
Training loss at step:  11  is  275.8448
Training loss at step:  12  is  275.41818
Training loss at step:  13  is  275.08792
Training loss at step:  14  is  274.8322
Training loss at step:  15  is  274.634
Training loss at step:  16  is  274.4808
Training loss at step:  17  is  274.36185
Training loss at step:  18  is  274.2699
Training loss at step:  19  is  274.19867
Training loss at step:  20  is  274.14355
Training loss at step:  21  is  274.10077
Training loss at step:  22  is  274.06766
Training loss at step:  23  is  274.0421
Training loss 

In [31]:
## Testing loss at every 10 step.

In [32]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:X_test_n, y_:y_test})
    
    if epoch % 10 == 0:
        print ('Testing loss at step: ', epoch, ' is ', train_loss)

Testing loss at step:  0  is  57.884933
Testing loss at step:  10  is  56.986824
Testing loss at step:  20  is  56.917156
Testing loss at step:  30  is  56.911755
Testing loss at step:  40  is  56.911324
Testing loss at step:  50  is  56.9113
Testing loss at step:  60  is  56.911312
Testing loss at step:  70  is  56.911274
Testing loss at step:  80  is  56.91129
Testing loss at step:  90  is  56.911312


### Get the shapes and values of W and b

In [33]:
W_optimum=sess.run(W)


In [34]:
b_optimum=sess.run(b)

In [35]:
print(W_optimum)

[[1.154436 ]
 [1.1584178]
 [1.1425873]
 [1.1679258]]


In [36]:
print(b_optimum)

[2.3118074]


### Model Prediction on 1st Examples in Test Dataset

In [37]:
import numpy as np

In [38]:
prediction = np.matmul(np.float32(X_test[0]),W_optimum) + b_optimum
print(prediction)

[252.11195]


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

##### Load the libraries

In [39]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### Load the given Iris data using pandas (Iris.csv)

In [40]:
df_iris = pd.read_csv('Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [41]:
# Print a concise summary of a DataFrame
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


In [42]:
# get the count
df_iris['Species'].value_counts()

Iris-setosa        50
Iris-virginica     50
Iris-versicolor    50
Name: Species, dtype: int64

In [43]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df_iris['SpeciesID'] = labelencoder.fit_transform(df_iris["Species"])

In [44]:
df_SpeciesID = df_iris[['SpeciesID']]

In [45]:
df_iris = pd.get_dummies(df_iris, columns=['SpeciesID'])

In [46]:
df_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,SpeciesID_0,SpeciesID_1,SpeciesID_2
0,1,5.1,3.5,1.4,0.2,Iris-setosa,1,0,0
1,2,4.9,3.0,1.4,0.2,Iris-setosa,1,0,0
2,3,4.7,3.2,1.3,0.2,Iris-setosa,1,0,0
3,4,4.6,3.1,1.5,0.2,Iris-setosa,1,0,0
4,5,5.0,3.6,1.4,0.2,Iris-setosa,1,0,0


In [47]:
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 9 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
SpeciesID_0      150 non-null uint8
SpeciesID_1      150 non-null uint8
SpeciesID_2      150 non-null uint8
dtypes: float64(4), int64(1), object(1), uint8(3)
memory usage: 7.5+ KB


### Splitting the data into feature set and target set

In [48]:
y = df_iris[['SpeciesID_0', 'SpeciesID_1','SpeciesID_2']]

In [49]:
X = df_iris.drop(['Id','Species', 'SpeciesID_0', 'SpeciesID_1','SpeciesID_2'],axis=1)

### Divide the dataset into Training and test (70:30)

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [52]:
model = Sequential()

In [53]:
model.add(Dense(64, input_dim=4, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [54]:
# comile the model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Model Training 

In [55]:
model.fit(X_train,y_train, validation_data=(X_test, y_test),epochs=100,batch_size=1)

Instructions for updating:
Use tf.cast instead.
Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 1s 5ms/step - loss: 1.0450 - acc: 0.5238 - val_loss: 0.6811 - val_acc: 0.7111
Epoch 2/100
105/105 [==============================] - 0s 1ms/step - loss: 0.6245 - acc: 0.7143 - val_loss: 0.5289 - val_acc: 0.6222
Epoch 3/100
105/105 [==============================] - 0s 1ms/step - loss: 0.4715 - acc: 0.7810 - val_loss: 0.5354 - val_acc: 0.7111
Epoch 4/100
105/105 [==============================] - 0s 1ms/step - loss: 0.4177 - acc: 0.8000 - val_loss: 0.5122 - val_acc: 0.6222
Epoch 5/100
105/105 [==============================] - 0s 1ms/step - loss: 0.3704 - acc: 0.8476 - val_loss: 0.3197 - val_acc: 0.9778
Epoch 6/100
105/105 [==============================] - 0s 1ms/step - loss: 0.3306 - acc: 0.8476 - val_loss: 0.2929 - val_acc: 0.9556
Epoch 7/100
105/105 [==============================] - 0s 1ms/step - loss: 0.2925 - acc: 0.8667 - val_loss: 0

105/105 [==============================] - 0s 2ms/step - loss: 0.1547 - acc: 0.9238 - val_loss: 0.2669 - val_acc: 0.8667
Epoch 60/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1415 - acc: 0.9333 - val_loss: 0.2430 - val_acc: 0.8889
Epoch 61/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1729 - acc: 0.9238 - val_loss: 0.0705 - val_acc: 1.0000
Epoch 62/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1108 - acc: 0.9524 - val_loss: 0.2192 - val_acc: 0.8889
Epoch 63/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1368 - acc: 0.9429 - val_loss: 0.2496 - val_acc: 0.8889
Epoch 64/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1261 - acc: 0.9238 - val_loss: 0.1643 - val_acc: 0.9333
Epoch 65/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1040 - acc: 0.9238 - val_loss: 0.0948 - val_acc: 0.9556
Epoch 66/100
105/105 [==============================] - 0s 1ms/step - loss:

### Model Prediction

In [56]:
# predict the model
y_predict=model.predict_classes(X_test)
y_predict

array([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 2, 0, 0, 1, 2,
       2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2,
       1], dtype=int64)

### Save the Model

In [57]:
model.save('iris.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [58]:
model = Sequential()

In [59]:
model.add(Dense(64,input_dim=4, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [60]:
# comile the model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [61]:
# fit the model
model.fit(X_train,y_train, validation_data=(X_test, y_test),epochs=100,batch_size=1)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 1s 7ms/step - loss: 0.7274 - acc: 0.6571 - val_loss: 0.5663 - val_acc: 0.6000
Epoch 2/100
105/105 [==============================] - 0s 2ms/step - loss: 0.4461 - acc: 0.7619 - val_loss: 0.4695 - val_acc: 0.6222
Epoch 3/100
105/105 [==============================] - 0s 1ms/step - loss: 0.4216 - acc: 0.7333 - val_loss: 0.3740 - val_acc: 0.7556
Epoch 4/100
105/105 [==============================] - 0s 1ms/step - loss: 0.4179 - acc: 0.7810 - val_loss: 0.4611 - val_acc: 0.6222
Epoch 5/100
105/105 [==============================] - 0s 2ms/step - loss: 0.3642 - acc: 0.8286 - val_loss: 0.3471 - val_acc: 0.7556
Epoch 6/100
105/105 [==============================] - 0s 2ms/step - loss: 0.3390 - acc: 0.8286 - val_loss: 0.5493 - val_acc: 0.6222
Epoch 7/100
105/105 [==============================] - 0s 2ms/step - loss: 0.3274 - acc: 0.8381 - val_loss: 0.4615 - val_acc: 0.7111
Epoch 8/100
105/105 [===

Epoch 62/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1549 - acc: 0.9429 - val_loss: 0.0868 - val_acc: 0.9778
Epoch 63/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1252 - acc: 0.9714 - val_loss: 0.0568 - val_acc: 1.0000
Epoch 64/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1328 - acc: 0.9524 - val_loss: 0.0698 - val_acc: 0.9778
Epoch 65/100
105/105 [==============================] - 0s 2ms/step - loss: 0.1350 - acc: 0.9524 - val_loss: 0.1403 - val_acc: 0.9333
Epoch 66/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1271 - acc: 0.9714 - val_loss: 0.1812 - val_acc: 0.9111
Epoch 67/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1193 - acc: 0.9619 - val_loss: 0.0852 - val_acc: 0.9778
Epoch 68/100
105/105 [==============================] - 0s 1ms/step - loss: 0.1396 - acc: 0.9524 - val_loss: 0.2597 - val_acc: 0.8889
Epoch 69/100
105/105 [==============================] - 0s 1ms

In [62]:
# predict the model
y_predict=model.predict_classes(X_test)
y_predict

array([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 2, 0, 0, 2, 2,
       2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2,
       1], dtype=int64)

In [63]:
# by adding 2 hidden layer performance decreases.. 
# this might be due to overfitting and dataset might be linerally seperable and don't need hidden layer